

# Ephaptic Coupling #

### Import stuff

In [1]:
%pylab inline

import numpy as np
import sys
if sys.version < '3':
    from urllib2 import urlopen
else:    
    from urllib.request import urlopen
import ssl
from warnings import warn
import zipfile
import os
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.collections import PolyCollection
from os.path import join
import LFPy
import neuron
from neuron import h

Populating the interactive namespace from numpy and matplotlib


### Define morphology and cell parameters

In [2]:

cellParameters = {
    'morphology' : 'morphology/purkinje.hoc',
    'cm' : 0.65,                      
    'Ra' : 120,                
    'v_init' : -62, 
    'passive' : True,           # turn on NEURONs passive mechanism for all sections
    'nsegs_method' : None,      # spatial discretization method
    'dt' : 2**-6,               # simulation time step size
    'tstart' : -200,               # start time of simulation
    'tstop' : 400,               # stop simulation
    'celsius': 34,
    'pt3d' : True,
    'extracellular': True,
    'passive_parameters' : {'g_pas' : 1/120236, 'e_pas' : -62},
}

### Create generator_cell

In [3]:
# delete old sections from NEURON namespace
LFPy.cell.neuron.h("forall delete_section()")

# initialize cell instance, using the LFPy.Cell class
generator_cell = LFPy.Cell(**cellParameters)
# align cell
generator_cell.set_rotation(x=4.99, y=-4.33, z=0)

# create synapse
synapse_parameters = {
    'idx' : generator_cell.get_closest_idx(x=0, y=0, z=0),  # place sinapse at soma coordinates (should always be 0,0,0 - but better check for different morphos)
    'e' : 15,                    # reversal potential
    'syntype' : 'ExpSyn',        # synapse type
    'tau1' : 0.5,                # synaptic time constant
    'tau2' : 2,
    'weight' : 0.005,            # synaptic weight
    'record_current' : True,     # record synapse current
}

synapse = LFPy.Synapse(generator_cell, **synapse_parameters)
synapse.set_spike_times(np.array([10.]))

	1 
	30 
	0 
	0 
	0 
	0 
Purkinje cell morphology defined

	1 


### Create grid electrode (for contour plot) and point electrode (for simple plots)

In [ ]:
# place electrodes at each segment of the cell
# define parameters for extracellular recording electrode, using optional method
# x,y,z are coordinates of contacts (from the grid in xz-plane over which we calculate local field potentials)
electrodeParameters = {
    'sigma' : 0.3,                  # extracellular conductivity
    'x' : generator_cell.xmid,      # + distance of the electrodes to the receptor cell
    'y' : generator_cell.ymid,
    'z' : generator_cell.zmid,
    'method' : 'soma_as_point',     # sphere source soma segment
    'N' : np.array([[0, 1, 0]]*generator_cell.xmid.size),      # surface normals
    'r' : 2.5,                      # contact site radius
    'n' : 20,                       # datapoints for averaging
}

# create a grid of measurement locations, in (mum)
X, Z = np.mgrid[-300:300:20, -150:1200:10]
Y = np.zeros(X.shape)

# define electrode parameters
grid_electrode_parameters = {
    'sigma' : 0.3,             # extracellular conductivity
    'x' : X.flatten(),         # electrode requires 1d vector of positions
    'y' : Y.flatten(),
    'z' : Z.flatten(),
    'verbose': True,
}

# define electrode parameters
point_electrode_parameters = {
    'sigma' : 0.3,                    # extracellular conductivity
    'x' : np.array([130., 220.]),     # positions of the first(blue) and second(green) electrode
    'y' : np.array([   0.,    0.]),
    'z' : np.array([   0.,  700.]),
}

# run simulation, electrode object argument in cell.simulate
generator_cell.simulate(rec_imem=True)

# create electrode objects
grid_electrode = LFPy.RecExtElectrode(generator_cell,**grid_electrode_parameters)
point_electrode = LFPy.RecExtElectrode(generator_cell,**point_electrode_parameters)

# calculate LFPs
grid_electrode.calc_lfp()
point_electrode.calc_lfp()

### Plot generator and receptor cells

In [ ]:
# plot LFP blue electrode [0]
plt.axes([.15,.10,.25,.25])
plt.plot(generator_cell.tvec,point_electrode.LFP[0]*1e6,color='blue', clip_on=False)
plt.ylabel(r'$\Phi_2(\mathbf{r},t)$ (nV)')
plt.xlabel(r'$t$ (ms)')
plt.axis('tight')
ax = plt.gca()
for loc, spine in ax.spines.items():
    if loc in ['right', 'top']:
        spine.set_color('none')            
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

# plot LFP green electrode
plt.axes([.15,.375,.25,.25])
plt.plot(generator_cell.tvec,point_electrode.LFP[1]*1e6,color='green', clip_on=False)
plt.ylabel(r'$\Phi_1(\mathbf{r}, t)$ (nV)')
plt.axis('tight')
ax = plt.gca()
for loc, spine in ax.spines.items():
    if loc in ['right', 'top']:
        spine.set_color('none')            
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.set_xticklabels([])

# plot synapse electrode
plt.axes([.15,.65,.25,.25])
plt.plot(generator_cell.tvec,synapse.i*1E3, color='red', clip_on=False)
plt.ylabel(r'$i_\mathrm{syn}(t)$ (pA)')
plt.axis('tight')
ax = plt.gca()
for loc, spine in ax.spines.items():
    if loc in ['right', 'top']:
        spine.set_color('none')            
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.set_xticklabels([])

# optionally save figure
# plt.savefig('contour_plot.pdf', dpi=300)

In [ ]:
def ctime(t,ax=None,vmin=-15,vmax=-5):
    
    ax = plt.gca() if ax == None else ax

    #LFP = np.max(np.abs(grid_electrode.LFP),1).reshape(X.shape)  
    LFP = np.abs(grid_electrode.LFP[:,t]).reshape(X.shape)  

    im = ax.contour(X, Z, np.log10(LFP), 20,
               cmap='inferno', vmin=vmin, vmax=vmax,  #'viridis', 'plasma', 'inferno', 'magma', 'cividis' https://matplotlib.org/tutorials/colors/colormaps.html#mycarta-banding
               zorder=-2)
    #cbar = fig.colorbar(im, cax=cax)
    #cbar.set_label('$|\Phi(\mathbf{r}, t)|$ (nV)')
    #cbar.outline.set_visible(False)
    
    # plot points where the synapse is
    ax.plot(generator_cell.xmid[generator_cell.synidx],generator_cell.zmid[generator_cell.synidx], 'o', ms=5,
        markeredgecolor='k',
        markerfacecolor='r')

    # plot points where the LFP is been recorded
    color_vec = ['blue','green']
    for i in range(2):
        ax.plot(point_electrode_parameters['x'][i], point_electrode_parameters['z'][i],'o',ms=3, markeredgecolor='k',
            markerfacecolor=color_vec[i])
    
    # plot morphology of generator_cell
    zips = []
    for x, z in generator_cell.get_idx_polygons():
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips,
                             edgecolors='none',
                             facecolors='k')
    ax.add_collection(polycol)

    # plot morphology of receptor_cell
    shift = 100          # shift of receptor_cell in x dimension
    zips = []
    for x, z in generator_cell.get_idx_polygons():
        zips.append(list(zip(x+shift, z)))
    polycol = PolyCollection(zips,
                             edgecolors='none',
                             facecolors= 'orange')
    ax.add_collection(polycol)

    ax.set_ylim(-250,1200)
    ax.set_xlim(-300,300)
    ax.axis('off')
    
    #print(np.any(min(np.log10(LFP[:,:]))))
    
fig,axes = plt.subplots(2,3,figsize=(16,8))
times = [600,641,650,670,1800,24000]
for t,ax in zip(times,axes.ravel()):
    ctime(t,ax=ax)
    ax.set_title('Time %.1f ms'%(t*2**-6))

In [ ]:
plt.figure(figsize=(1,5))
plt.imshow(np.linspace(-10,-5,100).reshape(-1,1),vmin=-10,vmax=-5,cmap='inferno', aspect='auto')
plt.gca().set_axis_off()






In [ ]:
#Histograma para definir os limites vmin e vmax
#plt.hist(np.log10(np.abs(grid_electrode.LFP.ravel())),30)